In [4]:
import asyncio
from hume import HumeStreamClient
from hume.models.config import LanguageConfig

samples = [
    "Mary had a little lamb,",
    "Its fleece was white as snow.",
    "Everywhere the child went,",
    "The little lamb was sure to go."
]

async def main():
    client = HumeStreamClient(hume_api)
    config = LanguageConfig()
    async with client.connect([config]) as socket:
        for sample in samples:
            result = await socket.send_text(sample)
            emotions = result["language"]["predictions"][0]["emotions"]
            print(emotions)

# 检查是否已存在事件循环
loop = asyncio.get_event_loop()
if loop.is_running():
    # 如果事件循环已经在运行，则使用已有的循环
    loop.create_task(main())
else:
    # 如果没有事件循环在运行，则正常使用 asyncio.run
    asyncio.run(main())


In [9]:
import asyncio
import json
from hume import HumeStreamClient
from hume.models.config import LanguageConfig

hume_api = "bGKD8LrwTrKIHS6BQFVV8p8rrImljoCTiAQik7Yp2Phk3GiV" #需要换成你的API密钥
top_n = 5  # 保留的情感数量
all_emotions= []  # 创建一个空列表来存储所有的情感数据


file_path = input("Enter the path to the text file: ") #读取断句文件
with open(file_path, 'r', encoding='utf-8') as file:
    sentences = [line.strip() for line in file] #删除空行

print("Total number of sentences:", len(sentences)) #打印所有句子数量

try:
    start = int(input("Enter the start index (1-based): ")) #输入需要读取的首句
    end = int(input("Enter the end index (1-based): ")) #输入需要读取的末句
    
    if not 1 <= start <= len(sentences) or not 1 <= end <= len(sentences) or start > end:
        raise ValueError("Invalid range!") #若输入数字不符合要求则报错

    samples = sentences[start - 1:end] #替换samples
except ValueError as ve: #反之，都报错
    print("Invalid input:", ve)
except IndexError:
    
    print("Index out of range!")
except Exception as e:
    print("An error occurred:", str(e))

async def main(top_n,all_emotions):
    client = HumeStreamClient(hume_api)
    config = LanguageConfig()
    async with client.connect([config]) as socket:
        for sample in samples:
            result = await socket.send_text(sample)
            emotions = result["language"]["predictions"][0]["emotions"]
            # 使用top_n变量来控制保留的情感数量
            top_emotions = sorted(emotions, key=lambda x: x['score'], reverse=True)[:top_n]
            all_emotions.append({sample: top_emotions})  # 将每个样本的top N情感数据添加到列表中
            print(top_emotions)
    
    # 将所有情感数据写入JSON文件
    with open('top_emotions.json', 'w') as f:
        json.dump(all_emotions, f, ensure_ascii=False, indent=4)  # 确保使用非ASCII字符和格式化输出

# 检查是否已存在事件循环
loop = asyncio.get_event_loop()
if loop.is_running():
    # 如果事件循环已经在运行，则使用已有的循环
    loop.create_task(main(top_n,all_emotions))  # 传入你想要的截取数量，例如10
else:
    # 如果没有事件循环在运行，则正常使用 asyncio.run
    asyncio.run(main(top_n,all_emotions))  # 同样传入截取数量

Total number of sentences: 10288


In [19]:
# 打印所有情感数据,这些数据被组织成了一个列表由字典组成，每个字典包含了一个样本的top N情感数据
all_emotions

[{'1. PART ONE Chapter 1 It was a bright cold day in April, and the clocks were striking thirteen.': [{'name': 'Boredom',
    'score': 0.16840356588363647},
   {'name': 'Annoyance', 'score': 0.09857569634914398},
   {'name': 'Calmness', 'score': 0.09760010242462158},
   {'name': 'Confusion', 'score': 0.08876815438270569},
   {'name': 'Concentration', 'score': 0.0757199078798294}]},
 {'2. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.': [{'name': 'Boredom',
    'score': 0.20710931718349457},
   {'name': 'Calmness', 'score': 0.19328643381595612},
   {'name': 'Awkwardness', 'score': 0.07403583079576492},
   {'name': 'Contentment', 'score': 0.07400946319103241},
   {'name': 'Confusion', 'score': 0.06857922673225403}]},
 {'3. The hallway smelt of boiled cabbage and old rag mats.': [{'name': 'Boredom',
  